In [0]:
%%capture
import numpy as np
import tensorflow as tf
import tflearn
import random
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')
#...

# Preprocessando a base de dados

In [0]:
database = [
    {"classe": "horario", "sentencas": (
        "qual vai ser o horário de robótica?",
        "Alguém que faz IC com Abner(circuitos) tem como perguntar a ele qual o horário?",
        "Alguém sabe o horário das disciplinas optativas?",
        "Robótica esse período, foi em quais horários?",
        "Qual horário de calculo 2 nesse período?",
        "Qual o horário de IP2", "Que horas começa a aula de TC?"
    )},
    {"classe": "duvida","sentencas": (
        "O que você pode fazer?", "O que você faz?", "Em que você pode me ajudar?", 
        "Me diga o que pode fazer", "Qual sua função?", "para que você serve?", "Do que você é capaz?"
    )},
    {"classe": "ementa", "sentencas": (
        "Tem como enviar ementa da disciplina?", "Ementa de teoria", 
        "Qual a ementa de algoritmos", "Assunto da cadeira de algebra", 
        "Plano de aula de arquitetura", "Assunto das aulas de compiladores"
    )},
    {"classe": "pre_requisito", "sentencas": (
        "Sabe se precisa de algum pré requisito para IA?", "SO tem pre requisito?", 
        "Qual pre-requisito para prog2", "Lista de pre requisitos para projetão",
        "quais a cadeiras que preciso ter pago para pagar Sistemas Operacionais", 
        "Quais os pre-requisitos", "o que tenho que pagar antes de banco de dados?"
    )},
    {"classe": "carga_horária", "sentencas": (
        "Sem carga horária não serve.",
        "Ele disse que cumpriu a carga horária",
        "Se der certificado e lá tiver escrito a carga horária",
        "Qual a carga horária", "Quanto tempo", "Qual a ch?", "Quantas horas"
    )},
    {"classe": "sala", "sentencas": (
        "discreta I qual a sala?",
        "Alguém sabe a sala de circuitos?",
        "qual a sala de Prog2", "onde é a aula de calculo?",
    )},
    {"classe": "professor", "sentencas": (
        "Quem ta dando discreta?", "qual professor de pensamento computacional",
        "quem eh o professor de IP2", "quem dá matematica discreta", 
        "qual o professor de algebra", "qual o nome do professor de prog2"
    )},
    {"classe": "estagio", "sentencas": (
        "Como funciona eso", "como cortar eso", "trabalho dispensa eso", "estagio corta eso",
        "como posso dispensar ESO", "trabalho carteira asinada dispensa estágio"
    )}
]

# Importando as bibliotecas


In [0]:
words = []
classes = []
documents = []
ignore_words = ['?', ':']

for intent in database:
    for pattern in intent['sentencas']:
        # Tokenizando cada padrão
        w = nltk.word_tokenize(pattern)
        # Lista geral de padrões
        words.extend(w)
        # Lista de padrões com a classe correspondente
        documents.append((w, intent['classe']))
        # Adiciona a classe a lista de classes
        if intent['classe'] not in classes:
            classes.append(intent['classe'])

# Aplicando stemming, palavras em minúsculo e ignorando as palavras definidas
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Removendo duplicatas
classes = sorted(list(set(classes)))

In [0]:
# To determine which version you're using:
!pip show tensorflow

# For the current version: 
!pip install --upgrade tensorflow

# For a specific version:
!pip install tensorflow==1.2

# For the latest nightly build:
!pip install tf-nightly

In [0]:
print (len(documents), "documentos")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

50 documentos
8 classes ['carga_horária', 'duvida', 'ementa', 'estagio', 'horario', 'pre_requisito', 'professor', 'sala']
130 unique stemmed words ['(', ')', ',', '.', '2', 'a', 'abn', 'ajud', 'alg', 'algebr', 'algoritmo', 'alguém', 'ant', 'arquitetur', 'asinad', 'assunto', 'aul', 'aula', 'banco', 'cadeir', 'cadeira', 'calculo', 'capaz', 'carg', 'carteir', 'certificado', 'ch', 'circuito', 'com', 'começ', 'como', 'compilad', 'computac', 'cort', 'cumpriu', 'da', 'dado', 'dando', 'das', 'de', 'der', 'dig', 'disciplin', 'disciplina', 'discret', 'dispens', 'diss', 'do', 'dá', 'e', 'eh', 'el', 'em', 'ement', 'envi', 'escrito', 'eso', 'ess', 'estagio', 'estágio', 'faz', 'foi', 'funcion', 'função', 'hora', 'horár', 'horário', 'i', 'ia', 'ic', 'ip2', 'list', 'lá', 'matematic', 'me', 'ness', 'nom', 'não', 'o', 'ond', 'operaciona', 'optativa', 'os', 'pag', 'pago', 'par', 'pensamento', 'pergunt', 'período', 'plano', 'pod', 'posso', 'pre', 'pre-requisito', 'pre-requisitos', 'precis', 'preciso', 'pr

In [0]:
training = []
output = []

# Conjunto de trainamento, bloco de palavras para cada sentença
for doc in documents:
    # Iniciando o bloco
    block = []
    # Lista de palavras para cada padrão
    pattern_words = doc[0]
    # Aplica stemming para cada palavra
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # Cria o bloco de palavras
    for w in words:
        block.append(1) if w in pattern_words else block.append(0)
    # Saída é '0' para cada classe e '1' para a classe corrent
    output_row = list([0] * len(classes))
    output_row[classes.index(doc[1])] = 1
    training.append([block, output_row])

# Embaralha as caracteristicas e converte para um array numpy
random.shuffle(training)
training = np.array(training)

# Cria listas de treino
train_x = list(training[:,0])
train_y = list(training[:,1])

# Construindo a Rede Neural

In [0]:
# Parâmetros

# Dados
X = train_x
Y = train_y
# Número de neurônios por camada
num_neurons_per_layer = [8, 8]
# Número de camadas internas
num_inner_layers = len(num_neurons_per_layer)
# Tipo de ativação. Outros algoritmos: http://tflearn.org/activations/
activation = 'softmax'
# Número de epocas
num_epoch = 1000

# fim dos parâmetos 

# Limpa as definições
tf.reset_default_graph()
# Cria a camada de entrada
net = tflearn.input_data(shape=[None, len(X[0])])
# Cria as camadas internas
for l in range(num_inner_layers):
    net = tflearn.fully_connected(net, num_neurons_per_layer[l])
# Cria a camada de saída
net = tflearn.fully_connected(net, len(Y[0]), activation=activation)
# aplica regressão
net = tflearn.regression(net)

# Define o modelo e configura o tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Começa o treinamento (apply gradient descent algorithm)
model.fit(X, Y, n_epoch=num_epoch, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 6999  | total loss: 0.51336 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.51336 - acc: 0.9307 -- iter: 48/50
Training Step: 7000  | total loss: 0.46668 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.46668 - acc: 0.9376 -- iter: 50/50
--


In [0]:
# Limpa as definições
tf.reset_default_graph()
# Cria a camada de entrada
net = tflearn.input_data(shape=[None, len(X[0])])
# Cria as camadas internas
for l in range(num_inner_layers):
    net = tflearn.fully_connected(net, num_neurons_per_layer[l])
# Cria a camada de saída
net = tflearn.fully_connected(net, len(Y[0]), activation=activation)
# aplica regressão
net = tflearn.regression(net)

# Define o modelo e configura o tensorboard
model2 = tflearn.DNN(net)

model2.load('./model.tflearn')

# Salva os dados de treino

In [0]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'database': database, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

# Funções para tratar o input do usuário

In [0]:
def clean_up_sentence(sentence):
    # Tokeniza o padrão
    sentence_words = nltk.word_tokenize(sentence)
    # Aplica stemming para cada palavra
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Retorna um array representando a sentença em: 0 or 1 para 
# cada palavra do bloco (words) que existe na sentença
def bow(sentence, words, show_details=False):
    # Tokeniza o padrão
    sentence_words = clean_up_sentence(sentence)
    # Inicializa o bloco de palavras
    block = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                block[i] = 1
                if show_details:
                    print ("found in block: %s" % w)
    return(np.array(block))

# Recebe a intenção e um limiar para filtrar as predições
# Retorna uma lista com a probabilidade de cada intenção
def get_prob_intentions(sentence, threshold=0.6):
    treated = bow(sentence, words)
    prediction = model.predict([treated]).flatten()
    sorted_intent_predict = sorted(zip(prediction, classes), key=lambda e: e[0], reverse=True)
    return list(filter(lambda e: e[0] > threshold, sorted_intent_predict))

# Busca no banco as respostas para uma determinada intenção
def get_responses_from_database(database, intention):
    obj = {}
    for e in database:
        if e['tag'] == intention:
            return e['responses']
    return []

from random import choice
# Retorna uma resposta baseado na intenção
def get_response(database, sentence):
    prob_intentions = get_prob_intentions(sentence)
    if len(prob_intentions) == 0: return DEFAULT_RESPONSE
    highest = prob_intentions[0]
    confidence, intention = highest
    responses = get_responses_from_database(database, intention)
    return choice(responses)

# Testando o bot

In [0]:
bot_name = 'Bot'
print('Fale tchau para sair!\n')
while True:
    pergunta = input('Você: ')
    if pergunta.lower() == 'tchau':
        break
    print(f'{bot_name}: {get_response(database, pergunta)}')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
